In [1]:
import numpy as np
from numpy import array
import os
import re
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
#from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

### Cargamos las imagenes del dataset

In [2]:
# Obtenemos la dirección donde está nuestro dataset
dire = os.getcwd()[:-3]
dirname = os.path.join(dire, 'data\\test')
imgpath = dirname + os.sep
 
images = []
directories = []
dircount = []
prevRoot=''
cant=0

# Cargamos las imágenes
print("leyendo imagenes de ",imgpath,"\n")
 
for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            if len(np.shape(image)) == 3:
                images.append(image)
            else:
                cant-=1
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print("Cargado:",root)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)

dircount = dircount[1:]
dircount[len(directories)-1]=dircount[len(directories)-1]+1

print('Carpetas leidas:',len(directories))
print('Total de imagenes:',sum(dircount))

leyendo imagenes de  C:\Users\PC\OneDrive\Documentos\Proyectos CC\Proyecto\ProyectoRN\data\test\ 

Cargado: C:\Users\PC\OneDrive\Documentos\Proyectos CC\Proyecto\ProyectoRN\data\test\air hockey
Cargado: C:\Users\PC\OneDrive\Documentos\Proyectos CC\Proyecto\ProyectoRN\data\test\ampute football
Cargado: C:\Users\PC\OneDrive\Documentos\Proyectos CC\Proyecto\ProyectoRN\data\test\archery
Cargado: C:\Users\PC\OneDrive\Documentos\Proyectos CC\Proyecto\ProyectoRN\data\test\arm wrestling
Cargado: C:\Users\PC\OneDrive\Documentos\Proyectos CC\Proyecto\ProyectoRN\data\test\axe throwing
Cargado: C:\Users\PC\OneDrive\Documentos\Proyectos CC\Proyecto\ProyectoRN\data\test\balance beam
Cargado: C:\Users\PC\OneDrive\Documentos\Proyectos CC\Proyecto\ProyectoRN\data\test\barell racing
Cargado: C:\Users\PC\OneDrive\Documentos\Proyectos CC\Proyecto\ProyectoRN\data\test\baseball
Cargado: C:\Users\PC\OneDrive\Documentos\Proyectos CC\Proyecto\ProyectoRN\data\test\basketball
Cargado: C:\Users\PC\OneDrive\Docume

### Clasificando y etiquetando las entradas

In [4]:
X = array(images, dtype=np.uint8)/255
etiquetas = []
indice = 0
for cantidad in dircount:
    for i in range(cantidad):
        etiquetas.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(etiquetas))
 
deportes=[]
indice = 0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    deportes.append(name[len(name)-1])
    indice=indice+1

y = np.array(etiquetas).astype('float32')
#X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
 
# Find the unique numbers from the train labels
classes = np.unique(y)
print('Número total de clases: ', len(classes))

Cantidad etiquetas creadas:  499
0 air hockey
1 ampute football
2 archery
3 arm wrestling
4 axe throwing
5 balance beam
6 barell racing
7 baseball
8 basketball
9 baton twirling
10 bike polo
11 billiards
12 bmx
13 bobsled
14 bowling
15 boxing
16 bull riding
17 bungee jumping
18 canoe slamon
19 cheerleading
20 chuckwagon racing
21 cricket
22 croquet
23 curling
24 disc golf
25 fencing
26 field hockey
27 figure skating men
28 figure skating pairs
29 figure skating women
30 fly fishing
31 football
32 formula 1 racing
33 frisbee
34 gaga
35 giant slalom
36 golf
37 hammer throw
38 hang gliding
39 harness racing
40 high jump
41 hockey
42 horse jumping
43 horse racing
44 horseshoe pitching
45 hurdles
46 hydroplane racing
47 ice climbing
48 ice yachting
49 jai alai
50 javelin
51 jousting
52 judo
53 lacrosse
54 log rolling
55 luge
56 motorcycle racing
57 mushing
58 nascar racing
59 olympic wrestling
60 parallel bar
61 pole climbing
62 pole dancing
63 pole vault
64 polo
65 pommel horse
66 rings
67 

In [8]:
train_X,test_X,train_Y,test_Y = train_test_split(X,y)
print("Tamaño del conjunto de entrenamiento:", len(train_Y))
print("Tamaño del conjunto de prueba:", len(test_Y))

# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)
 
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2)
 
print(len(train_X),len(valid_X),len(train_label),len(valid_label))

Tamaño del conjunto de entrenamiento: 374
Tamaño del conjunto de prueba: 125
299 75 299 75


In [12]:
INIT_LR = 1e-3
epochs = 6
batch_size = 64
inShape = np.shape(images[0])
 
sport_model = Sequential()
sport_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=inShape))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(MaxPooling2D((2, 2),padding='same'))
sport_model.add(Dropout(0.5))
 
sport_model.add(Flatten())
sport_model.add(Dense(32, activation='linear'))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(Dropout(0.5)) 
sport_model.add(Dense(nClasses, activation='softmax'))
 
sport_model.summary()
 
sport_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adagrad(learning_rate=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 112, 112, 32)      0         
                                                                 
 flatten_3 (Flatten)         (None, 401408)            0         
                                                                 
 dense_6 (Dense)             (None, 32)                12845088  
                                                      

In [14]:
sport_train_dropout = sport_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))
 
# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar
sport_model.save("sports_mnist.h5py")

Epoch 1/6
5/5 [==============================] - 5s 981ms/step - loss: 4.5989 - accuracy: 0.0067 - val_loss: 4.6039 - val_accuracy: 0.0000e+00
Epoch 2/6
5/5 [==============================] - 5s 964ms/step - loss: 4.6016 - accuracy: 0.0100 - val_loss: 4.6226 - val_accuracy: 0.0133
Epoch 3/6
5/5 [==============================] - 5s 967ms/step - loss: 4.5990 - accuracy: 0.0067 - val_loss: 4.6132 - val_accuracy: 0.0000e+00
Epoch 4/6
5/5 [==============================] - 5s 1s/step - loss: 4.5798 - accuracy: 0.0167 - val_loss: 4.6279 - val_accuracy: 0.0267
Epoch 5/6
5/5 [==============================] - 5s 985ms/step - loss: 4.5862 - accuracy: 0.0435 - val_loss: 4.5963 - val_accuracy: 0.0133
Epoch 6/6
5/5 [==============================] - 5s 971ms/step - loss: 4.5807 - accuracy: 0.0134 - val_loss: 4.5994 - val_accuracy: 0.0000e+00
INFO:tensorflow:Assets written to: sports_mnist.h5py\assets


In [16]:
test_eval = sport_model.evaluate(test_X, test_Y_one_hot)
 
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

ValueError: in user code:

    File "C:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1366, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1356, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1349, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1305, in test_step
        self.compiled_loss(
    File "C:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\PC\AppData\Local\Programs\Python\Python3